In [1]:
import fmpm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn
import skimage.io
import math
import random

import torchvision.transforms

In [2]:
random.seed(1)
image_dir = 'data/10x'
labels_file = 'data/me.csv'

DATA = fmpm.prep_data(pd.read_csv(labels_file), image_dir)

transforms = torchvision.transforms.Compose([
                            torchvision.transforms.ToPILImage(),
                            torchvision.transforms.RandomRotation((-180,180)),
                            torchvision.transforms.CenterCrop((325)),
                            torchvision.transforms.ToTensor()
                                      ])

train_data = fmpm.tenX_dataset(DATA, 'data/10x', transform =transforms)

/Users/Will/GradSchool/CHEME/Project/FmPM/fmpm/will/fmpm.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  identification[i] = False
/Users/Will/GradSchool/CHEME/Project/FmPM/fmpm/will/fmpm.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  identification[i] = True


In [3]:
def normalize_image(im):
    #Could as try nn.function.normalize
    count = 0
    for channel in im:
        mean = torch.mean(channel).item()
        std = torch.std(channel).item()
        im[count] = (channel - mean) / std
        count += 1
    
    return im

In [4]:
def test_normalize_image():
    test_im = torch.Tensor([[[10, 0, 10],
                             [5, 0, 5]],
                           [[100, 0, 100],
                            [10, 0, 10]]
                           ])
    print(normalize_image(test_im))
    
    
test_normalize_image()

tensor([[[ 1.1180, -1.1180,  1.1180],
         [ 0.0000, -1.1180,  0.0000]],

        [[ 1.2857, -0.7443,  1.2857],
         [-0.5413, -0.7443, -0.5413]]])


In [5]:
for samp in range(len(train_data)):
    #plt.figure(samp)
    im = train_data[samp]['image']
    #plt.imshow(pimp_my_image(im))
    print(im)
    if samp is 0: break

tensor([[[0.9529, 0.9451, 0.9725,  ..., 0.9804, 0.9608, 0.9686],
         [0.9725, 0.9686, 0.9608,  ..., 0.9882, 0.9686, 0.9686],
         [0.9725, 0.9686, 0.9686,  ..., 0.9882, 0.9804, 0.9725],
         ...,
         [0.9765, 0.9804, 0.9843,  ..., 0.9961, 0.9882, 0.9922],
         [0.9961, 0.9804, 0.9922,  ..., 0.9922, 0.9882, 0.9922],
         [0.9961, 0.9961, 0.9961,  ..., 0.9922, 0.9922, 0.9804]],

        [[0.4196, 0.4118, 0.4000,  ..., 0.4275, 0.4275, 0.4353],
         [0.4392, 0.4196, 0.4000,  ..., 0.4275, 0.4196, 0.4196],
         [0.4392, 0.4275, 0.4196,  ..., 0.4275, 0.4196, 0.4118],
         ...,
         [0.4392, 0.4196, 0.4196,  ..., 0.4588, 0.4510, 0.4667],
         [0.4588, 0.4392, 0.4392,  ..., 0.4510, 0.4667, 0.4667],
         [0.4588, 0.4588, 0.4431,  ..., 0.4510, 0.4667, 0.4667]],

        [[0.2471, 0.2078, 0.1961,  ..., 0.2706, 0.2353, 0.2353],
         [0.2235, 0.2235, 0.2078,  ..., 0.2353, 0.2627, 0.2627],
         [0.2471, 0.2471, 0.2353,  ..., 0.2471, 0.2471, 0.

#### The CNN archetecture

In [6]:
class LeNet(torch.nn.Module):
    def __init__(self):
        """
        Initializes CNN. Here we just define layer shapes that we call in the forward func
        """
        super().__init__()

        self.conv1 = torch.nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 5)
                
        #Convultion layer 2. See above
        self.conv2 = torch.nn.Conv2d(in_channels = 6, 
                               out_channels = 12, 
                               kernel_size = 5)
        
        self.fc_1 = torch.nn.Linear(39 * 39 * 12, 256)
        self.fc_2 = torch.nn.Linear(256, 2)
            
    def forward(self, x):
        """
        Function that performs all the neural network forward calculation i.e.
        takes image data from the input of the neural network to the output
        """
        
        x = self.conv1(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size = 2)
        x = torch.nn.functional.leaky_relu(x)
        x = self.conv2(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size = 4)
        x = torch.nn.functional.leaky_relu(x)
        x = x.view(x.shape[0], -1)  
        x = self.fc_1(x) 
        x = torch.nn.functional.leaky_relu(x)
        x = self.fc_2(x)    
        
        return x

In [7]:
#Declaring iterator. The thing that will loop through our dataset.

train_data = fmpm.tenX_dataset(DATA, 'data/10x', transform =transforms)
BATCH_SIZE = 10
train_iterator = torch.utils.data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)


#Instancing model, loss criteria, device to perform calculations on, and optimizer.
model = LeNet()


criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=.005)

#Telling the model and loss function to do math on whatever device is
model = model.to(device)
criterion = criterion.to(device)

In [8]:
def calculate_accuracy(y_pred, y):
    #argmax returns the indices of the maximum value of all elements in the input tensor.
    #We do this along the column of 2 element tensors returning a single column of predictions
    #Then we compare to our target, convert to a number, calculate the average for the batch
    acc = ((y_pred.argmax(dim=1) == y).float().mean())
    return acc

In [9]:
def train(model, iterator, optimizer, criterion, device):
    """
    Training loop. Takes data through NN calculates loss and adjusts NN. Repeat
    """
    epoch_loss = 0
    epoch_acc = 0
    #Need to add logic to skip iteration if image is None
    for sample in iterator:  
        image = sample['image'].to(device)
        isPlasticRaw = sample['plastic'].to(device)
        image = normalize_image(image)
        optimizer.zero_grad()      
        y_pred = model(image)
        isPlastic = isPlasticRaw.argmax(dim=1)
        loss = criterion(y_pred, isPlastic)
        acc = calculate_accuracy(y_pred, isPlastic)
        loss.backward()    
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator) , epoch_acc / len(iterator), y_pred, isPlasticRaw

In [ ]:
#Here the model is actually trained
EPOCHS = 50
for epoch in range(EPOCHS+1):
    train_loss, train_acc, y_pred, target = train(model, train_iterator, optimizer, criterion, device)
    print(f'EPOCH: {epoch}, acc: {train_acc}, loss: {train_loss}')
    if epoch % 5 is 0:
        print(y_pred)
        print(target)


EPOCH: 0, acc: 0.7415770583095089, loss: 1.449638243223871
tensor([[ 0.5219, -0.2852],
        [ 0.7293, -0.3940],
        [ 0.5198, -0.2876],
        [ 0.4769, -0.2511],
        [ 0.5242, -0.2876],
        [ 0.5560, -0.3177],
        [ 0.4954, -0.2652],
        [ 0.5019, -0.2706],
        [ 9.2358, -6.7210]], grad_fn=<AddmmBackward>)
tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0]])
EPOCH: 1, acc: 0.8025089598471119, loss: 0.5159527063369751
EPOCH: 2, acc: 0.802508956001651, loss: 0.5042894965698642
EPOCH: 3, acc: 0.8096774182012004, loss: 0.5112192068369158
EPOCH: 4, acc: 0.8060931851786952, loss: 0.47697674002378215
EPOCH: 5, acc: 0.8537634322720189, loss: 0.47753952203258393
tensor([[ 1.6012, -3.0040],
        [ 7.3741, -6.0684],
        [ 1.1949, -2.5606],
        [ 4.5312, -4.6729],
        [ 1.1902, -2.5735],
        [ 1.2004, -2.5662],
        [ 1.1984, -2.5630],
        [ 1.4624, -2.

In [ ]:
def test_dataset_class():
    filename = '../tests/test_assets/test_labels_unclean.csv'
    image_dir = '../tests/test_assets/test_images'
    
    #Eventually just read in already cleaned labels sheet
    labels = prep_data(pd.read_csv(filename), image_dir)
    
    
    transform = None
    test_dataset = tenX_dataset(labels, image_dir, transform)
    
    #len() check
    length = len(test_dataset)
    expect = 5
    assert length is expect, f'10x dataset length method failed. Got {length}, should be {expect}'
    
    #get_item() check
    samples = []
    keys = ['image', 'plastic','shape','color']
    for i in range(len(test_dataset)):
        assert test_dataset[i]['image'] is not None, 'Got NoneType instead of image'
        isP = test_dataset[i]['plastic']
        assert math.isclose(0,isP) or math.isclose(1,isP), f'plastic not 0 or 1, instead is {isP}'
        length = len(test_dataset[i]['shape'])
        assert length is 4, f'length of shape array not 4, instead is {length}'
        samples.append(test_dataset[i])
        
    shape = samples[0]['shape']
    assert math.isclose(shape[2], 1),  f'wrong shape first image, is {shape}'
    color = samples[0]['color']
    assert math.isclose(color[0], 1), f'wrong color first image, is {color}'
    assert math.isclose(samples[4]['plastic'], 1), 'wrong plastic id 3rd image'
    
test_dataset_class()

In [ ]:
test_dataset_class

In [ ]:
labels.to_csv('try')

In [ ]:
labels.head()

In [ ]:
labels_frame['isPlastic'].astype(bool).describe()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

output = torch.randn(10, 2).float()
target = torch.FloatTensor(10).uniform_(0, 2).long()

loss = criterion(output, target)
print(output)
print(output.size())
print(target)
print(target.size())